In [1]:
import base64
import requests

# OpenAI API Key
api_key = "xxxxx"

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [2]:
import pandas as pd

base_path = "clear/apartment_images/"
gt_path = 'ground_truth.csv'

# convert list of images to path
def image_list(folder_id, image_string):
    images = []
    array = image_string.split(';')
    for item in array:
        image_url = base_path + str(folder_id) + '/image_' + item + '.jpg'
        images.append(image_url)
    return images


In [26]:
# get training ids
import random

heating_electric_panels_ids = [4, 5, 19, 22, 23, 31, 44]
len(heating_electric_panels_ids)*0.6

4.2

In [27]:
heating_electric_panels_training_sample = random.sample(heating_electric_panels_ids, 5) 
print(heating_electric_panels_training_sample)

[31, 23, 4, 22, 19]


In [28]:
heating_electric_storage_ids = [13, 30, 45]
heating_electric_storage_training_sample = random.sample(heating_electric_storage_ids, 2) 
print(heating_electric_storage_training_sample)

[13, 30]


In [29]:
heating_underfloor_ids = [0, 10, 11, 14, 16, 17, 18, 21, 25, 29, 33, 35, 37, 38, 40, 41]
len(heating_underfloor_ids)*0.6

9.6

In [30]:
heating_underfloor_training_sample = random.sample(heating_underfloor_ids, 10) 
print(heating_underfloor_training_sample)

[25, 17, 35, 16, 41, 33, 38, 40, 21, 14]


In [32]:
heating_warm_air_ids = [3, 34, 36]
heating_warm_air_training_sample = random.sample(heating_warm_air_ids, 2) 
print(heating_warm_air_training_sample)

[3, 34]


In [33]:
heating_water_rads_ids = [1, 2, 6, 7, 8, 9, 12, 15, 20, 24, 26, 27, 28, 32, 39, 42, 43, 46]
len(heating_water_rads_ids)*0.6

10.799999999999999

In [34]:
heating_water_rads_training_sample = random.sample(heating_water_rads_ids, 11) 
print(heating_water_rads_training_sample)

[39, 43, 20, 42, 8, 1, 9, 32, 6, 24, 27]


In [35]:
# Combine all lists and check for duplicates, there shouldn't be!
training_samples_combined = heating_electric_panels_training_sample + heating_underfloor_training_sample + heating_warm_air_training_sample + heating_water_rads_training_sample
duplicates = [item for item in set(training_samples_combined) if training_samples_combined.count(item) > 1]
print("Duplicates:", duplicates)

Duplicates: []


In [36]:
# Sort them by increasing id
training_sample = sorted(training_samples_combined)
print(training_sample)
if len(training_sample) != 28:
    print("insufficient training sample", len(training_sample))
else:
    print("training sample length", len(training_sample))

[1, 3, 4, 6, 8, 9, 14, 16, 17, 19, 20, 21, 22, 23, 24, 25, 27, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43]
training sample length 28


In [37]:
from openai import OpenAI
client = OpenAI(api_key=api_key)
def get_completion(prompt):
    response = client.chat.completions.create(
      model="gpt-4o-2024-08-06", #"gpt-4o",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt,
            },
          ],
        }
      ],
      max_tokens=4000,
    )
    # print(response)
    return response.choices[0].message.content

In [44]:
# choose 3 buildings to extract feature from

# random pick one from water rads
item1 = random.sample(heating_water_rads_ids, 1)[0]
print('item1 id:', item1)

item1 id: 12


In [39]:
# randomly pick one from electric panels
item2 = random.sample(heating_electric_panels_ids, 1)[0]
print('item2 id:', item2)

item2 id: 4


In [41]:
# randomly pick one from warm air
item3 = random.sample(heating_warm_air_ids, 1)[0]
print('item3 id:', item3)

item3 id: 34


In [45]:
custom_prompt = "Your task is to provide a detailed label of every visible feature in the images relating to heating type that will help determine the type of heating used whether it is underfloor heating, water radiators, electric heaters, electric storage heaters or warm air from vents. List 50 visible features that are significant for determining the type of heating used in the apartment. Don't explain the label."
image_type = 'heating image' # kwh uses age. change this for age, window etc. this is used to determine which images to use

def llm_get_features(images):

    prompt = f"""

    You are a surveyor. You are given a set of images that belong to the same building.
    
    {custom_prompt}
    
    The building is located in UK. Return the features as a list.
    
    """

    content = [
            {
              "type": "text",
              "text": prompt
            }
          ]

    for image in images:
        base64_image = encode_image(image)
        content.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        })

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": content
        }
      ],
      "max_tokens": 5000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # print(response.text)
    print(response.json()['choices'][0]['message']['content'])


def extract_features(row_id):
    df = pd.read_csv(gt_path)
    raw_image_list = df.loc[row_id][image_type]
    print('\n\n\n---row', row_id, df.loc[row_id]['Address'], df.loc[row_id]['Thesquare Building URL'])
    llm_get_features(image_list(row_id, raw_image_list))

In [46]:
extract_features(item1)




---row 12 Chelsea Cloisters , Sloane Avenue , SW3 3DT https://www.thesqua.re/london/serviced-apartments/chelsea-serviced-apartments-by-mysquare-1429547768
1. Radiators
2. Thermostatic radiator valves
3. Pipes connected to radiators
4. Digital thermostat
5. Boiler controls
6. Wall vent grilles
7. Baseboard heaters
8. Electric heater units
9. Storage heater units
10. Underfloor heating controls
11. Hydronic pipes
12. Heating vents
13. Ducts for warm air
14. Radiator covers
15. Radiator piping
16. Thermometer on wall
17. Electric socket with heater switch
18. Gas meter around heaters
19. Floor type (tiles for underfloor heating)
20. Ventilation systems
21. Control panels for heating
22. Ceiling fans absent or present
23. Heater timers
24. Programmable thermostats
25. Heater safety grilles
26. Powered down heaters
27. Flame ignition points
28. Infrared heaters
29. Gas heaters
30. Water heating tanks
31. Combi boiler
32. Smoke detectors nearby heaters
33. Heater remote control
34. Air co

In [47]:
extract_features(item2)




---row 4 8 Timber Street , 8 Timber Street , EC4V 3PA https://www.thesqua.re/london/serviced-apartments/sir-john-lyon-house-serviced-apartment-by-thesquare-868937704
1. Wall-mounted heater
2. Absence of visible radiators
3. Electric heater control panel
4. Vent openings
5. Placement of heater near furniture
6. No visible pipes
7. Sleek heater design
8. Floor type: hardwood
9. Minimal space between floor and heater
10. Heater size and shape
11. Shiny reflective heater surface
12. Presence of thermostat control
13. Heater located high on wall
14. Modern interior design compatibility
15. Lack of traditional radiator valves
16. Absence of underfloor heating controls
17. Use of electrical outlets near heater
18. Compact heater design
19. Wall material around heater
20. Floating furniture arrangement
21. Nearby outlet placement
22. Interior door and heater layout
23. Heater integration with decor
24. Absence of heating ducts
25. Heater proximity to couch
26. Single heating source
27. Heat

In [48]:
extract_features(item3)




---row 34 Apartment 19, 1 Lambs Passage, London EC1Y8AB , 1 Lambs Passage , EC1Y8AB https://www.thesqua.re/london/serviced-apartments/lambs-passage-in-city-of-london-by-thesquare-1207657713
1. Wall-mounted thermostat
2. Radiator placement
3. Radiator pipework
4. Radiator design
5. Underfloor heating control panel
6. Floor vents
7. Wall grills
8. Baseboard heaters
9. Electric wall-mounted heater
10. Storage heater units
11. Presence of fan-assisted heaters
12. Electric panel heaters
13. Underfloor heating mats
14. Pipe insulation visible
15. Heating ducts
16. Ventilation controls
17. Wall-mounted storage heater
18. Thermostatic radiator valves
19. Electric fireplace
20. Heater timers
21. Wall-mounted heater controls
22. Heat pump units
23. Boiler controls
24. Hydronic heating manifolds
25. Radiant floor heating evidence
26. Ceiling heating units
27. HVAC system controls
28. Central heating boiler presence
29. Hot water cylinder cabinet
30. Thermostat wiring
31. Boiler flue placement


In [50]:
features_1 = [
  "Radiators",
  "Thermostatic radiator valves",
  "Pipes connected to radiators",
  "Digital thermostat",
  "Boiler controls",
  "Wall vent grilles",
  "Baseboard heaters",
  "Electric heater units",
  "Storage heater units",
  "Underfloor heating controls",
  "Hydronic pipes",
  "Heating vents",
  "Ducts for warm air",
  "Radiator covers",
  "Radiator piping",
  "Thermometer on wall",
  "Electric socket with heater switch",
  "Gas meter around heaters",
  "Floor type (tiles for underfloor heating)",
  "Ventilation systems",
  "Control panels for heating",
  "Ceiling fans absent or present",
  "Heater timers",
  "Programmable thermostats",
  "Heater safety grilles",
  "Powered down heaters",
  "Flame ignition points",
  "Infrared heaters",
  "Gas heaters",
  "Water heating tanks",
  "Combi boiler",
  "Smoke detectors nearby heaters",
  "Heater remote control",
  "Air conditioning units",
  "Solar heating panels",
  "Storage heater input/output dials",
  "Electric heating strips",
  "Wall-mounted heater units",
  "Room-by-room heating",
  "Heat pump systems",
  "Heated towel racks",
  "Weatherproofing strips on windows",
  "Curtains as insulation",
  "Insulated walls",
  "External wall vents",
  "Metal radiator fins",
  "Boiler pressure gauges",
  "Air filtration systems",
  "Pellet stove exhausts",
  "Visible chimney ducts"
]

In [51]:
features_2 = [
  "Wall-mounted heater",
  "Absence of visible radiators",
  "Electric heater control panel",
  "Vent openings",
  "Placement of heater near furniture",
  "No visible pipes",
  "Sleek heater design",
  "Floor type: hardwood",
  "Minimal space between floor and heater",
  "Heater size and shape",
  "Shiny reflective heater surface",
  "Presence of thermostat control",
  "Heater located high on wall",
  "Modern interior design compatibility",
  "Lack of traditional radiator valves",
  "Absence of underfloor heating controls",
  "Use of electrical outlets near heater",
  "Compact heater design",
  "Wall material around heater",
  "Floating furniture arrangement",
  "Nearby outlet placement",
  "Interior door and heater layout",
  "Heater integration with decor",
  "Absence of heating ducts",
  "Heater proximity to couch",
  "Single heating source",
  "Heater's proximity to windows",
  "Absence of storage heater vents",
  "Heater placement near seating area",
  "Projected heat direction",
  "Heater width",
  "Wall space usage",
  "Lower heat panel",
  "Wall cavity space for heater",
  "Heater silhouette",
  "Minimalist heating design",
  "Adjustable heater height",
  "Floorboard spacing",
  "Absence of radiator flow controls",
  "Wall heater integration",
  "Heater operational buttons",
  "Seamless wall finish",
  "Modern fixture alignment",
  "Direction of furnishings",
  "Heater symmetry with room",
  "Placement relative to doorway",
  "Direct-to-wall heater fixings",
  "Heater beneath window",
  "Contrast of heater with flooring",
  "Temperature control access"
]

In [52]:
features_3 = [
  "Wall-mounted thermostat",
  "Radiator placement",
  "Radiator pipework",
  "Radiator design",
  "Underfloor heating control panel",
  "Floor vents",
  "Wall grills",
  "Baseboard heaters",
  "Electric wall-mounted heater",
  "Storage heater units",
  "Presence of fan-assisted heaters",
  "Electric panel heaters",
  "Underfloor heating mats",
  "Pipe insulation visible",
  "Heating ducts",
  "Ventilation controls",
  "Wall-mounted storage heater",
  "Thermostatic radiator valves",
  "Electric fireplace",
  "Heater timers",
  "Wall-mounted heater controls",
  "Heat pump units",
  "Boiler controls",
  "Hydronic heating manifolds",
  "Radiant floor heating evidence",
  "Ceiling heating units",
  "HVAC system controls",
  "Central heating boiler presence",
  "Hot water cylinder cabinet",
  "Thermostat wiring",
  "Boiler flue placement",
  "Chimney for coal or wood stoves",
  "Heat exchanger location",
  "Radiator shielding",
  "Wall vent proximity",
  "Pipe routing patterns",
  "Heat source noise",
  "Radiator fixing brackets",
  "Air conditioning units",
  "Room air vents",
  "Electric heater cord location",
  "Insulation panels",
  "Thermostat style (digital/analog)",
  "Floor material type (e.g., tiles for underfloor)",
  "Wall insulation visible",
  "Heating appliance labels",
  "Direct heating outlet proximity",
  "Central heating pump",
  "Air handling unit",
  "Heating system instruction manual"
]

In [53]:
raw_feature_list = features_1 + features_2 + features_3

prompt =  f"""
I have a list of features: {raw_feature_list}. First, remove duplicated items, including features semantically similar. Then cluster these features based on the type of feature. Aim to produce 8 clusters. 
"""

result = get_completion(prompt)
print(result)

To process your request, I will first remove duplicates and similar features, then cluster them into 8 categories based on their characteristics. Here's how the categorized clusters look:

1. **Radiators and Components**:
    - Radiators
    - Radiator covers
    - Radiator piping
    - Radiator placement
    - Radiator pipework
    - Radiator design
    - Radiator shielding
    - Radiator fixing brackets
    - Metal radiator fins

2. **Thermostats and Controls**:
    - Thermostatic radiator valves
    - Digital thermostat
    - Programmable thermostats
    - Thermometer on wall
    - Presence of thermostat control
    - Temperature control access
    - Wall-mounted thermostat
    - Thermostat wiring
    - Thermostat style (digital/analog)
    - Heating system instruction manual

3. **Boilers and Heating Systems**:
    - Boiler controls
    - Combi boiler
    - Central heating boiler presence
    - Boiler pressure gauges
    - Boiler flue placement
    - Central heating pump
    - Hydr

In [54]:
categories = f"""
1. **Radiators and Components**:
    - Radiators
    - Radiator covers
    - Radiator piping
    - Radiator placement
    - Radiator pipework
    - Radiator design
    - Radiator shielding
    - Radiator fixing brackets
    - Metal radiator fins

2. **Thermostats and Controls**:
    - Thermostatic radiator valves
    - Digital thermostat
    - Programmable thermostats
    - Thermometer on wall
    - Presence of thermostat control
    - Temperature control access
    - Wall-mounted thermostat
    - Thermostat wiring
    - Thermostat style (digital/analog)
    - Heating system instruction manual

3. **Boilers and Heating Systems**:
    - Boiler controls
    - Combi boiler
    - Central heating boiler presence
    - Boiler pressure gauges
    - Boiler flue placement
    - Central heating pump
    - Hydronic heating manifolds

4. **Heaters and Associated Features**:
    - Baseboard heaters
    - Electric heater units
    - Wall-mounted heater units
    - Electric wall-mounted heater
    - Electric fireplace
    - Infrared heaters
    - Gas heaters
    - Heat pump systems
    - Wall-mounted heater
    - Heater timers
    - Electric panel heaters

5. **Heating and Ventilation Infrastructure**:
    - Pipes connected to radiators
    - Heating vents
    - Ducts for warm air
    - Wall vent grilles
    - Floor vents
    - Vent openings
    - Wall grills
    - Ventilation systems
    - Pipe insulation visible
    - Heating ducts

6. **Room and Fixture Design**:
    - Absence of visible radiators
    - Sleek heater design
    - Compact heater design
    - Heater integration with decor
    - Modern interior design compatibility
    - Minimalist heating design
    - Modern fixture alignment
    - Heater symmetry with room

7. **Heating Augmentations and Accessories**:
    - Underfloor heating controls
    - Underfloor heating mats
    - Heated towel racks
    - Storage heater units
    - Storage heater input/output dials
    - Pellet stove exhausts
    - Ceiling fans absent or present

8. **Energy Efficiency and Safety**:
    - Weatherproofing strips on windows
    - Curtains as insulation
    - Insulated walls
    - Smoke detectors nearby heaters
    - Heater safety grilles
    - Insulation panels
"""

prompt =  f"""
Given this list {categories}, first clean the list to contain text only, then produce a python array, each subarray for each category.
"""

result = get_completion(prompt)
print(result)

To clean the list to contain text only and then produce a Python array with each subarray for each category, we would first remove any formatting characters and ensure the text is plain. Here is the cleaned list in Python array format:

```python
categories = [
    [
        "Radiators",
        "Radiator covers",
        "Radiator piping",
        "Radiator placement",
        "Radiator pipework",
        "Radiator design",
        "Radiator shielding",
        "Radiator fixing brackets",
        "Metal radiator fins"
    ],
    [
        "Thermostatic radiator valves",
        "Digital thermostat",
        "Programmable thermostats",
        "Thermometer on wall",
        "Presence of thermostat control",
        "Temperature control access",
        "Wall-mounted thermostat",
        "Thermostat wiring",
        "Thermostat style (digital/analog)",
        "Heating system instruction manual"
    ],
    [
        "Boiler controls",
        "Combi boiler",
        "Central heating boi